In [1]:
import awkward as ak
import uproot
import hist
import os
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.base import NanoEventsArray
from imp import reload #每次修改调用函数后可以更新
import correctionlib, rich
import matplotlib.pyplot as plt
import sys
from coffea.lumi_tools import LumiMask

/tmp/ipykernel_306166/3220351898.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload #每次修改调用函数后可以更新


In [98]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import time

def process_root_file(file_path, Tag, ABCD,Channel):
    try:
        # 记录开始时间
        start_time = time.time()
        # 从ROOT文件创建NanoEventsArray
        events = NanoEventsFactory.from_root(file_path, treepath="Events", schemaclass=NanoAODSchema).events()
        # 提取Muon值并进行必要的处理，例如计算Muon数量
        Event = events
        Electron = events.Electron
        Photon = events.Photon
        Muon = events.Muon

        #罗列selection
        if 'ele' in Channel:
            #electron
            ele_eta = (abs(Electron.eta) < 2.5) & ~((abs(Electron.eta) > 1.4442) & (abs(Electron.eta) < 1.566 ))
            ele_pt = (Electron.pt >= 20)
            ele_cutbased_id = (Electron.cutBased >= 3)
            lg_near, lg_dr = Photon.nearest(Electron, axis=1, return_metric=True) #find the nearest electron near each photon
        elif 'muon' in Channel:
            #muon
            muon_eta = (abs(Muon.eta) < 2.4) 
            muon_pt = (Muon.pt >= 20)
            muon_id_iso = (Muon.tightId) & (Muon.pfIsoId >= 4)    
            lg_near, lg_dr = Photon.nearest(Muon, axis=1, return_metric=True) #find the nearest electron near each photon     
        else: 
            print('Wrong Channel')

        #photon
        pho_pt = (Photon.pt >= 30)
        pho_eta = (abs(Photon.eta) < 2.5) & ~((abs(Photon.eta) > 1.4442) & (abs(Photon.eta) < 1.566))
        pho_drlg = lg_dr>0.5
        pho_pixel = ~Photon.pixelSeed
        pho_genpartFlav = (Photon.genPartFlav == 1)


        #联立选择条件
        if 'ele' in Channel:
            ele_selection = ele_eta & ele_pt & ele_cutbased_id
        elif 'muon' in Channel: 
            muon_selection = muon_eta & muon_pt & muon_id_iso
        else: 
            print('Wrong Channel!')
        photon_selection = pho_pt & pho_eta & pho_drlg 
        
        if 'data' in Tag:
            photon_selection = photon_selection  
        elif 'data' not in Tag and 'dy' not in Tag:
            photon_selection = photon_selection & pho_pixel & pho_genpartFlav 
        elif 'dy' in Tag:
            photon_selection = photon_selection & (~pho_pixel | ~pho_genpartFlav)
        else:
            print('Wrong sample!')


        #处理通过object挑选的事件 要求有两个electron，两个electron异号，最大的electron pt>=25
        if 'ele' in Channel:
            good_electron = Electron[ele_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_event = Event[(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_photon = Photon[photon_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            tem_lepton = good_electron
        elif 'muon' in Channel:
            good_muon = Muon[muon_selection][(ak.num(Muon[muon_selection])==2) & (ak.sum(Muon[muon_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Muon[muon_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_event = Event[(ak.num(Muon[muon_selection])==2) & (ak.sum(Muon[muon_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Muon[muon_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_photon = Photon[photon_selection][(ak.num(Muon[muon_selection])==2) & (ak.sum(Muon[muon_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Muon[muon_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]  
            tem_lepton = good_muon          
        else:
            print('Wrong channel!')

        #最终处理事例，调用函数，mass在质量窗内，且通过trigger
        #mass质量窗
        lepton_vectors = ak.zip(
        {
        "pt": tem_lepton["pt"],
        "eta": tem_lepton["eta"],
        "phi": tem_lepton["phi"],
        "mass": tem_lepton["mass"],
        },
        with_name="PtEtaPhiMLorentzVector",
        )
        mass_ll = (lepton_vectors[:, 0] + lepton_vectors[:, 1]).mass

        #photon id
        pt_id = (good_photon.vidNestedWPBitmap[:] & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 1) & 1) 
        scEta_id = (good_photon.vidNestedWPBitmap[:] >> 2 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 3) & 1) 
        HoE_id = (good_photon.vidNestedWPBitmap[:] >> 4 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 5) & 1) 
        siesie_id = (good_photon.vidNestedWPBitmap[:] >> 6 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 7) & 1) 
        chiso_id = (good_photon.vidNestedWPBitmap[:] >> 8 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 9) & 1) 
        neuiso_id = (good_photon.vidNestedWPBitmap[:] >> 10 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 11) & 1) 
        phoiso_id = (good_photon.vidNestedWPBitmap[:] >> 12 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 13) & 1)
        if ABCD == "A":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
        elif ABCD == "B":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
        elif ABCD == "C":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
        elif ABCD == "D":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
        else:
            print('Wrong ABCD value!')

        if 'ele' in Channel:
            selected_good_event = good_event[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_electron = good_electron[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_photon = good_photon[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            VARIABLES = {
                'photon_pt': ak.Array(selected_good_photon[:,0].pt),
                'photon_eta': ak.Array(selected_good_photon[:,0].eta),
                'ele1_pt':ak.Array(selected_good_electron[:,0].pt),
                'run':ak.Array(selected_good_event.run),
                # 添加更多的变量...
            }
        elif 'muon' in Channel:
            selected_good_event = good_event[(good_event.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_muon = good_muon[(good_event.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_photon = good_photon[(good_event.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            VARIABLES = {
                'photon_pt': ak.Array(selected_good_photon[:,0].pt),
                'photon_eta': ak.Array(selected_good_photon[:,0].eta),
                'muon1_pt':ak.Array(selected_good_muon[:,0].pt),
                'run':ak.Array(selected_good_event.run),
                # 添加更多的变量...
            }
        else:
            print('Wrong channel!')

        # 返回结果和用时
        if 'fidu' not in Channel:
            # 记录结束时间
            end_time = time.time()
            return VARIABLES, end_time - start_time
########------------------------Fiducial part--------------------------##############
        elif 'fidu' in Channel:
            #选择条件(注意只选择zg样本)
            if 'zg' not in key:
                print('Wrong fiducial sample, ZG only!')
            Event = selected_good_event
            Lepton = selected_good_event.GenDressedLepton
            Photon = selected_good_event.GenIsolatedPhoton
            if 'ele' in Channel:
                #electron
                lepton_eta = (abs(Lepton.eta) < 2.5) & ~((abs(Lepton.eta) > 1.4442) & (abs(Lepton.eta) < 1.566 ))
            elif 'muon' in Channel:
                #muon
                lepton_eta = (abs(Lepton.eta) < 2.4)
            else:
                print('Wrong fiducial channel!')
            lepton_pt = (Lepton.pt >= 20)
            lg_near, lg_dr = Photon.nearest(Lepton, axis=1, return_metric=True) #find the nearest electron near each photon       
            pho_pt = (Photon.pt >= 30)
            pho_eta = (abs(Photon.eta) < 2.5) & ~((abs(Photon.eta) > 1.4442) & (abs(Photon.eta) < 1.566))
            pho_drlg = lg_dr>0.5

            #联立选择条件
            lepton_selection = lepton_eta & lepton_pt
            photon_selection = pho_pt & pho_eta & pho_drlg

            bad_lepton_selection = ~lepton_eta | ~lepton_pt
            bad_photon_selection = ~pho_pt | ~pho_eta | ~pho_drlg

            #处理通过object挑选的事件 要求有两个electron，两个electron异号，最大的electron pt>=25
            good_lepton = Lepton[lepton_selection][(ak.num(Lepton[lepton_selection])==2) & (ak.sum(Lepton[lepton_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_event = Event[(ak.num(Lepton[lepton_selection])==2) & (ak.sum(Lepton[lepton_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_photon = Photon[photon_selection][(ak.num(Lepton[lepton_selection])==2) & (ak.sum(Lepton[lepton_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]

            bad_lepton = Lepton[bad_lepton_selection][(ak.num(Lepton[lepton_selection])!=2) | (ak.sum(Lepton[lepton_selection].pdgId,axis=1)!=0) | (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) < 25, False)) | (ak.num(Photon[photon_selection])!=1)]
            bad_event = Event[(ak.num(Lepton[lepton_selection])!=2) | (ak.sum(Lepton[lepton_selection].pdgId,axis=1)!=0) | (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) < 25, False)) | (ak.num(Photon[photon_selection])!=1)]
            bad_photon = Photon[bad_photon_selection][(ak.num(Lepton[lepton_selection])!=2) | (ak.sum(Lepton[lepton_selection].pdgId,axis=1)!=0) | (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) < 25, False)) | (ak.num(Photon[photon_selection])!=1)]
            #进一步加函数
            tem_lepton = good_lepton
            lepton_vectors = ak.zip(
            {
            "pt": tem_lepton["pt"],
            "eta": tem_lepton["eta"],
            "phi": tem_lepton["phi"],
            "mass": tem_lepton["mass"],
            },
            with_name="PtEtaPhiMLorentzVector",
            )
            mass_ll = (lepton_vectors[:, 0] + lepton_vectors[:, 1]).mass

            selected_good_event = good_event[((mass_ll >= 71) & (mass_ll <= 111))]
            selected_good_lepton = good_lepton[((mass_ll >= 71) & (mass_ll <= 111))]
            selected_good_photon = good_photon[((mass_ll >= 71) & (mass_ll <= 111))]

            selected_bad_event = bad_event[~((mass_ll >= 71) & (mass_ll <= 111))]
            selected_bad_lepton = bad_lepton[~((mass_ll >= 71) & (mass_ll <= 111))]
            selected_bad_photon = bad_photon[~((mass_ll >= 71) & (mass_ll <= 111))]
            if 'ele' in Channel:
                VARIABLES2 = {
                    'gen_photon_pt': ak.Array(selected_good_photon[:,0].pt),
                    'gen_photon_eta': ak.Array(selected_good_photon[:,0].eta),
                    'out_gen_photon_pt':ak.Array(selected_bad_photon[:,0].pt),
                    # 添加更多的变量...
                }
                VARIABLES.update(VARIABLES2)
            elif 'muon' in Channel:
                VARIABLES = {
                    'gen_photon_pt': ak.Array(selected_good_photon[:,0].pt),
                    'gen_photon_eta': ak.Array(selected_good_photon[:,0].eta),
                    'out_gen_photon_pt':ak.Array(selected_bad_photon[:,0].pt),
                    # 添加更多的变量...
                }
                VARIABLES.update(VARIABLES2)
            else:
                print('Wrong Fiducial channel!')
            # 记录结束时间
            end_time = time.time()
            return VARIABLES, end_time - start_time
        else:
            print('Do you want to calc fidu xsec or not?')
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None, 0


In [99]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import input_file 
reload(input_file)

# 假设你有一个字典，key是样本名，value是对应的ROOT文件路径列表
# samples_dict = input_file.input_file('test')
samples_dict = input_file.input_file('simple')# 1min


ABCD = 'A'
Channel = 'fidu_ele'#muon fidu_ele fidu_muon
# Channel = 'ele'#muon fidu_ele fidu_muon
max_workers = 50  # 最大并行核心数

# 所有文件的处理结果
all_results = {}

# 初始化进度条
progress = tqdm(total=sum(len(files) for files in samples_dict.values()), desc='Processing files', unit='file')

with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = {}
    for key, files in samples_dict.items():
        for file in files:
            # 向函数传递额外的参数Tag和ABCD
            future = executor.submit(process_root_file, file, key, ABCD, Channel)
            futures[future] = key
    
    for future in as_completed(futures):
        key = futures[future]
        try:
            result_dict, duration = future.result()  # result_dict 是我们从函数中返回的字典
            if result_dict is not None:
                if key not in all_results:
                    all_results[key] = []
                all_results[key].append(result_dict)  # 存储整个字典
                # 更新进度条
                progress.update(1)
                progress.set_postfix({'last_duration': f'{duration:.2f}s', 'active_workers': f'{executor._max_workers}'})  # 显示上一个文件处理用时和活动核心数
        except Exception as e:
            print(f"Error processing future for key {key}: {e}")
            progress.update(1)

progress.close()  # 关闭进度条

# 此时 all_results 中存储的是每个文件对应的字典
print(all_results)

Processing files:  67%|██████▋   | 2/3 [00:02<00:01,  1.04s/file, last_duration=2.20s, active_workers=50]

Error processing file /data/bond/muty/v12_2022/ZG/DYGto2LG-1Jets_MLL-50_PTG-200_TuneCP5_13p6TeV_amcatnloFXFX-pythia8/DYGto2LG-1Jets_MLL-50_PTG-200_TuneCP5_13p6TeV_amcatnloFXFX-pythia8_2022/231122_155830/0000/tree_11.root: in RegularArray attempting to get 693, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/cpu-kernels/awkward_RegularArray_getitem_next_array_regularize.cpp#L19)


Processing files:  67%|██████▋   | 2/3 [00:02<00:01,  1.41s/file, last_duration=2.20s, active_workers=50]

{'zg3_22': [{'photon_pt': <Array [195, 424, 366, 308, ... 215, 250, 226] type='252 * ?float32[parameters={...'>, 'photon_eta': <Array [-0.0722, -0.721, ... 0.44, -0.303] type='252 * ?float32[parameters={"__d...'>, 'ele1_pt': <Array [139, 256, 168, 25, ... 163, 228, 65] type='252 * float32[parameters={"__...'>, 'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='252 * uint32[parameters={"...'>, 'gen_photon_pt': <Array [194, 425, 369, 315, ... 214, 259, 224] type='250 * ?float32[parameters={...'>, 'gen_photon_eta': <Array [-0.0712, -0.721, ... 0.439, -0.302] type='250 * ?float32[parameters={"__...'>, 'out_gen_photon_pt': <Array [] type='0 * ?float32[parameters={"__doc__": "pt"}]'>}], 'zg2_22': [{'photon_pt': <Array [261, 127, 105, 149, ... 97.5, 106, 125] type='558 * ?float32[parameters=...'>, 'photon_eta': <Array [-0.0535, -0.125, ... 0.879, -0.0776] type='558 * ?float32[parameters={"_...'>, 'ele1_pt': <Array [53.5, 156, 68.6, ... 165, 98.9, 88.5] type='558 * float32[parameters

In [100]:
def merge_sample_data(sample_name, sample_data):
    try:
        # 初始化一个字典，用于存储当前样本的合并数据
        merged_data = {}

        # 获取第一个条目中的信息键
        info_keys = list(sample_data[0].keys())

        # 遍历每个信息键
        for info_key in info_keys:
            # 从 sample_data 中提取当前信息键的数据
            data_to_concatenate = [entry[info_key] for entry in sample_data]

            # 使用 ak.concatenate 将数据连接在一起
            merged_data[info_key] = ak.concatenate(data_to_concatenate)
        
        return merged_data
    except Exception as e:
        print(f"Error merging data for sample {sample_name}: {e}")
        return {}


In [101]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

# sample_names = ["dataC_ele","dataD_ele","zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#
# sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#,
sample_names = ["zg2_22","zg3_22","dy_22"]#,"dy""zg2", "zg3"
sample_names = ["zg2_22","zg3_22"]#,"dy""zg2", "zg3"


# 初始化进度条
progress = tqdm(total=len(sample_names), desc='Merging samples', unit='sample')

# 所有样本的合并结果
merged_results = {}

with ProcessPoolExecutor(max_workers=max_workers) as executor:
    # 提交每个样本的合并任务到进程池
    futures = {executor.submit(merge_sample_data, sample_name, all_results[sample_name]): sample_name for sample_name in sample_names}
    
    for future in as_completed(futures):
        sample_name = futures[future]
        try:
            result = future.result()
            merged_results[sample_name] = result
            # 更新进度条
            progress.update(1)
        except Exception as e:
            print(f"Error processing future for sample {sample_name}: {e}")
            progress.update(1)

progress.close()  # 关闭进度条

# 此时 merged_results 中存储的是每个样本合并后的结果
print(merged_results)


Merging samples: 100%|██████████| 2/2 [00:00<00:00,  6.10sample/s]

{'zg2_22': {'photon_pt': <Array [261, 127, 105, 149, ... 97.5, 106, 125] type='558 * ?float32[parameters=...'>, 'photon_eta': <Array [-0.0535, -0.125, ... 0.879, -0.0776] type='558 * ?float32[parameters={"_...'>, 'ele1_pt': <Array [53.5, 156, 68.6, ... 165, 98.9, 88.5] type='558 * float32[parameters={"_...'>, 'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='558 * uint32[parameters={"...'>, 'gen_photon_pt': <Array [259, 127, 108, 152, ... 97.6, 107, 124] type='556 * ?float32[parameters=...'>, 'gen_photon_eta': <Array [-0.053, -0.125, ... 0.878, -0.0778] type='556 * ?float32[parameters={"__...'>, 'out_gen_photon_pt': <Array [] type='0 * ?float32[parameters={"__doc__": "pt"}]'>}, 'zg3_22': {'photon_pt': <Array [195, 424, 366, 308, ... 215, 250, 226] type='252 * ?float32[parameters={...'>, 'photon_eta': <Array [-0.0722, -0.721, ... 0.44, -0.303] type='252 * ?float32[parameters={"__d...'>, 'ele1_pt': <Array [139, 256, 168, 25, ... 163, 228, 65] type='252 * float32[parameters={"_

In [102]:
merged_results

{'zg2_22': {'photon_pt': <Array [261, 127, 105, 149, ... 97.5, 106, 125] type='558 * ?float32[parameters=...'>,
  'photon_eta': <Array [-0.0535, -0.125, ... 0.879, -0.0776] type='558 * ?float32[parameters={"_...'>,
  'ele1_pt': <Array [53.5, 156, 68.6, ... 165, 98.9, 88.5] type='558 * float32[parameters={"_...'>,
  'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='558 * uint32[parameters={"...'>,
  'gen_photon_pt': <Array [259, 127, 108, 152, ... 97.6, 107, 124] type='556 * ?float32[parameters=...'>,
  'gen_photon_eta': <Array [-0.053, -0.125, ... 0.878, -0.0778] type='556 * ?float32[parameters={"__...'>,
  'out_gen_photon_pt': <Array [] type='0 * ?float32[parameters={"__doc__": "pt"}]'>},
 'zg3_22': {'photon_pt': <Array [195, 424, 366, 308, ... 215, 250, 226] type='252 * ?float32[parameters={...'>,
  'photon_eta': <Array [-0.0722, -0.721, ... 0.44, -0.303] type='252 * ?float32[parameters={"__d...'>,
  'ele1_pt': <Array [139, 256, 168, 25, ... 163, 228, 65] type='252 * float